In [1]:
import numpy as np
import pandas as pd
import yfinance as yfin
import matplotlib.pyplot as plt
from sdocks.consts import InvestmentData
from sdocks.performance import PerfMetrics
from sdocks.indicators import TechnicalIndicator
from plotly import graph_objects as go
from pandas_datareader import data as web

In [2]:
yfin.pdr_override()
# benchmark_data=web.DataReader(
#     'SPY',start='2015-12-31',end='2020-01-01'
# ).reset_index()

In [3]:
# data=pd.read_csv('AWU.csv')
data=web.DataReader(
    'META',start='2015-12-31',end='2020-01-01'
).reset_index()

eod_data=data[['Date','Close']]
eod_data.columns=['date','close']
dates=[str(v)[:10] for v in list(eod_data.date.values)]
prices=list(eod_data.close.values)

[*********************100%%**********************]  1 of 1 completed


In [4]:
investment_data=InvestmentData(dates, prices)
ti=TechnicalIndicator(investment_data)

In [5]:
short_sma=ti.sma(5)
long_sma=ti.sma(20)

In [ ]:
investment_data.

In [6]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(investment_data.dates),
        y=investment_data.prices,
        name='price of META'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(short_sma['dates']),
        y=short_sma['values'],
        name='MA(5)'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(long_sma['dates']),
        y=long_sma['values'],
        name='MA(20)'
    )
)
fig.show()

In [ ]:
investment_data=InvestmentData(dates, prices)
investment_data.get_return()
investment_data.get_log_return()
data=investment_data.get_data()

In [ ]:
benchmark_data=InvestmentData(
    [str(v)[:10] for v in benchmark_data['Date'].values], 
    list(benchmark_data['Close'].values)
)
benchmark_data.get_log_return()
benchmark_data=benchmark_data.get_data()

In [ ]:
data=investment_data.get_data()
data=data[data.date >= '2016-01-01']
perf=PerfMetrics(
    data,
    0,
    benchmark_log_return=benchmark_data.dropna()['log return'].values,
    drawdown_method='percent',
    calmer_years=10
)
payload=perf.get_payload()

In [ ]:
payload

In [ ]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(data.date.values),
        y=data.price.values,
        name='prices'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(
            [payload['max_drawdown']['trough_date'], payload['max_drawdown']['peak_date']]
        ),
        y=[
            payload['max_drawdown']['trough_price'], payload['max_drawdown']['peak_price']
        ],
        name=f"Max Drawdown: {payload['max_drawdown']['max_drawdown']*100:.2f} %"
    )
)
fig.update_layout(
    dict(
        title='AWU'
    )
)